In [93]:
import pandas as pd
import duckdb
import json

import geopandas as gpd
from shapely.geometry import Polygon

In [155]:
bandung = []
with open('bandung.json', 'r') as f:
    bandung = json.load(f)

bandung
    

[{'province': 'JAWA BARAT',
  'district': 'BANDUNG BARAT',
  'sub_district': 'LEMBANG',
  'village': 'PAGERWANGI',
  'border': [[107.6273, -6.8236],
   [107.6268, -6.8236],
   [107.6263, -6.8236],
   [107.6256, -6.8236],
   [107.6253, -6.8234],
   [107.6252, -6.8234],
   [107.625, -6.8234],
   [107.6247, -6.8233],
   [107.6246, -6.8233],
   [107.6244, -6.8232],
   [107.6243, -6.8236],
   [107.6242, -6.8239],
   [107.6242, -6.8241],
   [107.6241, -6.8241],
   [107.624, -6.8243],
   [107.6239, -6.8245],
   [107.6239, -6.8246],
   [107.6238, -6.8248],
   [107.6237, -6.8248],
   [107.6227, -6.8246],
   [107.6226, -6.8244],
   [107.6223, -6.8239],
   [107.6222, -6.8237],
   [107.6219, -6.8236],
   [107.6218, -6.8236],
   [107.6217, -6.8237],
   [107.6216, -6.8239],
   [107.6215, -6.824],
   [107.6214, -6.8244],
   [107.6213, -6.8247],
   [107.6213, -6.8251],
   [107.6218, -6.8257],
   [107.6219, -6.8258],
   [107.6219, -6.8259],
   [107.6219, -6.8261],
   [107.6219, -6.8264],
   [107.6219, 

In [146]:
def coords_to_polygon(coords):
    try:
        temp = []
        for item in coords:
            temp.append(tuple(item))
        return Polygon(temp)
        # Polygon expects list of tuples, so we need to convert nested lists
        # return Polygon([(coord[0], coord[1]) for coord in coords])
    except Exception as e:
        print(coords)
        print(e)
        return None
        

In [156]:
import json
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon

data = bandung


# Ubah data JSON menjadi DataFrame
df = pd.json_normalize(data)



In [165]:
df['geometry'] = df['border'].apply(coords_to_polygon)
df = df.drop(columns=['border'])
# Ubah DataFrame menjadi GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry='geometry')
gdf

,province,district,sub_district,village,geometry
0,JAWA BARAT,BANDUNG BARAT,LEMBANG,PAGERWANGI,"POLYGON ((107.6273 -6.8236, 107.6268 -6.8236, ..."
1,JAWA BARAT,BANDUNG BARAT,LEMBANG,KAYUAMBON,"POLYGON ((107.6411 -6.8135, 107.6405 -6.8135, ..."
2,JAWA BARAT,BANDUNG BARAT,LEMBANG,LEMBANG,"POLYGON ((107.6182 -6.8124, 107.6179 -6.8123, ..."
3,JAWA BARAT,BANDUNG BARAT,LEMBANG,CIKIDANG,"POLYGON ((107.6503 -6.773, 107.6502 -6.7729, 1..."
4,JAWA BARAT,BANDUNG BARAT,LEMBANG,CIKAHURIPAN,"POLYGON ((107.6158 -6.7596, 107.6152 -6.7595, ..."
...,...,...,...,...,...
160,JAWA BARAT,BANDUNG BARAT,SAGULING,JATI,"POLYGON ((107.3866 -6.8803, 107.3855 -6.8805, ..."
161,JAWA BARAT,BANDUNG BARAT,SAGULING,GIRIMUKTI,"POLYGON ((107.4275 -6.8847, 107.4273 -6.8845, ..."
162,JAWA BARAT,BANDUNG BARAT,SAGULING,BOJONGHALEUANG,"POLYGON ((107.4601 -6.8611, 107.4602 -6.8609, ..."
163,JAWA BARAT,BANDUNG BARAT,SAGULING,CIPANGERAN,"POLYGON ((107.407 -6.8656, 107.4064 -6.8654, 1..."


In [173]:
from shapely.ops import unary_union

In [174]:
gdf['geometry'] = gdf['geometry'].apply(lambda geom: geom.buffer(0))

In [175]:
gdf_kecamatan = gdf.dissolve(by='sub_district')

# Memperbaiki geometri yang tidak valid setelah penggabungan
gdf_kecamatan['geometry'] = gdf_kecamatan['geometry'].apply(lambda geom: unary_union([geom]))


In [176]:

# Simpan hasil poligon kecamatan ke file GeoJSON
gdf_kecamatan.to_file('kecamatan.geojson', driver='GeoJSON')


/Users/nasriadzlani/Workspaces/Research/venv/lib/python3.12/site-packages/pyogrio/raw.py:698: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  warnings.warn(


In [92]:
mongo_data = []
for i, item in enumerate(bandung):
    temps = ""
    # print(temps)
    mongo_data.append({
        "_id": i+1,
        "province": "West Java",
        "district": "Bandung Barat",
        "sub_district": "Padalarang",
        "village": item["village"],
        "border": {
            "type": "Polygon",
            "coordinates": item['border']
        }
    })
    

with open('bandung_mongo.json', 'w') as f:
    json.dump(mongo_data, f)